In [1]:
import numpy as np
import pandas as pd
import folium
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

## Toronto Neighbourhoods

In [2]:
result = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
content = result.content
soup = BeautifulSoup(content,'lxml')
table = soup.find_all('table')[0]

postal_codes = pd.DataFrame()

for tr in table.find_all('tr')[1:]:
    td = tr.find_all("td")

    postal_code = str(td[0].contents[0])

    borough = str(td[1].contents[0])
    start = borough.find('">')+2
    start
    end = borough.find('</a>')
    end
    borough = borough[start:end]
    # Replace 'or' with York
    if (borough == 'or'):
        borough = 'York'
    if (borough == 'ississaug'):
        borough = 'Mississauga'
        
    neighbourhood = str(td[2].contents[0])
    if (neighbourhood[0] == "<"):
        start = neighbourhood.find('">')+2
        start
        end = neighbourhood.find('</a>')
        end
        neighbourhood = neighbourhood[start:end]    
    if (neighbourhood[-1] == "\n"):
        start = 0
        end = neighbourhood.find('\n')
        neighbourhood = neighbourhood[start:end]
    
    row_df = pd.DataFrame([[postal_code, borough, neighbourhood]])
    postal_codes = pd.concat([postal_codes, row_df],ignore_index=True)

# Set column Names    
postal_codes.columns=['PostalCode','Borough','Neighbourhood']

# Remove 'Not assigned' boroughs
postal_codes = postal_codes[postal_codes['Borough']!='ot assigne']

# Re-index
postal_codes = postal_codes.reset_index(drop=True)

# If a cell has a borough but a Not assigned neighbourhood, then the neighbourhood will be the same as the borough
postal_codes.loc[postal_codes['Neighbourhood'] == 'Not assigned','Neighbourhood'] = postal_codes['Borough']

#Combine neighbourhoods from the same borough
postal_codes_combined = postal_codes.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
postal_codes = postal_codes[['PostalCode','Borough','Neighbourhood']]

postal_codes_combined=postal_codes_combined.reset_index()
postal_codes_combined = postal_codes_combined[['PostalCode','Borough','Neighbourhood']]


## Associate postal codes with coordinates

In [3]:
# Coordinates
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates

toronto_neighbourhoods = pd.concat([postal_codes_combined,coordinates],axis=1)
toronto_neighbourhoods = toronto_neighbourhoods[['PostalCode','Borough','Neighbourhood','Latitude','Longitude']]
toronto_neighbourhoods.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## FourSquare Integration

In [4]:
CLIENT_ID = 'XT3PLM1C0BXYHLRKF20DAUEINAF0Z0HPY2SECZSNATD4BSOJ' # your Foursquare ID
CLIENT_SECRET = 'JQVVUKAHE12LS0D4DVHBEVPKYUWXHSYMPPF1R3P55JT54QFV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## How many coffee places are within a km of each postal code?

In [6]:
LIMIT = 100
radius = 1000
query = "coffee"
shop_count = pd.DataFrame(columns=['PostalCode','ShopCount'])

for index, row in toronto_neighbourhoods.iterrows():
    #print(row['PostalCode'],row['Latitude'], row['Longitude'])
    neighbourhood_latitude = row['Latitude']
    neighbourhood_longitude = row['Longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    query,
    radius, 
    LIMIT
    )

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON
    if ((nearby_venues.count().count()) > 0):
        shop_count_temp = pd.DataFrame(data=[[row['PostalCode'],nearby_venues.count()[0]]], columns=['PostalCode','ShopCount'])
    else:
        shop_count_temp = pd.DataFrame(data=[[row['PostalCode'],0]], columns=['PostalCode','ShopCount'])
    shop_count = pd.concat([shop_count, shop_count_temp])          



In [7]:
shop_count = shop_count.reindex()
shop_count.set_index('PostalCode', inplace=True)

toronto_neighbourhoods = toronto_neighbourhoods.reindex()
toronto_neighbourhoods.set_index('PostalCode',inplace=True)

In [8]:

toronto_coffee_shops = pd.concat([shop_count,toronto_neighbourhoods], axis=1)
toronto_coffee_shops


,ShopCount,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,,
M1B,3,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,1,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,4,Scarborough,Woburn,43.770992,-79.216917
M1H,4,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,2,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,4,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,4,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,0,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476


## Map of Toronto

In [16]:
address = 'Toronto, ON'

latitude = 43.6532
longitude = -79.3832

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, shop_count, neighbourhood in zip(toronto_coffee_shops['Latitude'], toronto_coffee_shops['Longitude'], toronto_coffee_shops['ShopCount'], toronto_coffee_shops['Neighbourhood']):
    label = '{}, {}'.format(toronto_coffee_shops, shop_count)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=shop_count/10,
        popup=neighbourhood,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto